In [1]:
import pandas as pd
import numpy as np
import os, sys
import torch
import json
sys.path.append('../')
from models.utils import rocksdb_knobs_make_dict

In [2]:
PATH_internal = "../data/internal"
PATH_external = "../data/external"
PATH_knobs = "../data/rocksdb_conf"
wk_len = 16

In [3]:
knobs = rocksdb_knobs_make_dict(PATH_knobs)
knobs = pd.DataFrame(data=knobs['data'].astype(np.float32), columns=knobs['columnlabels'])
columns = knobs.columns
knobs.head(4)

,max_background_compactions,max_background_flushes,write_buffer_size,max_write_buffer_number,min_write_buffer_number_to_merge,compaction_pri,compaction_style,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,...,open_files,block_size,cache_index_and_filter_blocks,max_bytes_for_level_base,max_bytes_for_level_multiplier,target_file_size_base,target_file_size_multiplier,num_levels,memtable_bloom_size_ratio,compression_ratio
0,6.0,16.0,1304576.0,3.0,2.0,1.0,0.0,3.0,24.0,44.0,...,10000.0,9216.0,0.0,5242880.0,11.0,1800192.0,1.0,8.0,0.10,0.67
1,2.0,14.0,911360.0,6.0,2.0,0.0,0.0,7.0,18.0,55.0,...,100000.0,15360.0,1.0,3145728.0,10.0,1729536.0,1.0,5.0,0.15,0.50
2,8.0,6.0,1474560.0,6.0,1.0,2.0,0.0,5.0,18.0,48.0,...,1000000.0,6144.0,0.0,4194304.0,9.0,1429504.0,1.0,6.0,0.05,0.36
3,12.0,8.0,550912.0,7.0,1.0,3.0,0.0,4.0,18.0,35.0,...,10000.0,3072.0,1.0,2097152.0,11.0,900096.0,1.0,7.0,0.05,0.63


In [4]:
wk_len = 16
internal_dict = {}

pruned_im = pd.read_csv(os.path.join(PATH_internal, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(wk_len):
    im = pd.read_csv(os.path.join(PATH_internal, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
#     break
internal_dict[0].head(4)

,rocksdb.block.cache.miss COUNT,rocksdb.block.cache.hit COUNT,rocksdb.block.cache.add COUNT,rocksdb.block.cache.index.miss COUNT,rocksdb.block.cache.index.hit COUNT,rocksdb.block.cache.index.add COUNT,rocksdb.block.cache.index.bytes.insert COUNT,rocksdb.block.cache.data.miss COUNT,rocksdb.block.cache.data.hit COUNT,rocksdb.block.cache.data.add COUNT,...,rocksdb.bytes.decompressed_P99,rocksdb.bytes.decompressed_P100,rocksdb.bytes.decompressed_COUNT,rocksdb.bytes.decompressed_SUM,rocksdb.db.flush.micros_P50,rocksdb.db.flush.micros_P95,rocksdb.db.flush.micros_P99,rocksdb.db.flush.micros_P100,rocksdb.db.flush.micros_COUNT,rocksdb.db.flush.micros_SUM
index,,,,,,,,,,,,,,,,,,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,1296845,1172926,1213729,0,0,0,0,1296845,1172926,1213729,...,8383.000000,8383.0,1297251,10856979511,21652.173913,30615.000000,30615.0,30615.0,44,1015663
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,1304820,4192080,1256074,1582,2701880,1582,5078096,1303238,1490200,1254492,...,14659.000000,14659.0,1304820,19074632333,18135.593220,21979.661017,26561.0,26561.0,63,1074316
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,1407632,2195141,1315590,0,0,0,0,1407632,2195141,1315590,...,6578.256132,12483.0,1407897,8847109704,29660.714286,45820.000000,45820.0,45820.0,78,2692570
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,1497592,7007880,1252738,974,4003817,974,6993856,1496618,3004063,1251764,...,4385.648850,9451.0,1497592,4718848784,24646.616541,32387.969925,34773.0,34773.0,208,5221234


In [5]:
external_dict = {}
for wk in range(wk_len):
    ex = pd.read_csv(os.path.join(PATH_external, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
#     break
external_dict[0].head(4)

,TIME,RATE,WAF,SA
index,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,21.8,4.77,8.7,72.17
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,25.9,4.02,8.9,55.83
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,52.0,2.00,7.5,34.21
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,53.6,1.94,10.5,56.94


## Test Train
- train: 16000, test: 4000
- target: 0 th workload

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader
from models.network import RocksDBDataset, SingleNet#, EncoderRNN, DecoderRNN
from models.train import train, valid#, trainRNN, validRNN

In [7]:
knobOneHot = np.load('../data/knobsOneHot.npy')
internal_m = internal_dict[2] # similar datasets
external_m = external_dict[2] # similar datasets
knobOneHot.shape, internal_m.shape, external_m.shape

((20000, 3327), (20000, 148), (20000, 4))

In [8]:
X_tr, X_te, Im_tr, Im_te, y_tr, y_te, knob_tr, knob_te = \
            train_test_split(knobOneHot, internal_m, external_m, knobs, test_size=0.2, random_state=24)
X_tr.shape, X_te.shape, Im_tr.shape, Im_te.shape, y_tr.shape, y_te.shape, knob_tr.shape, knob_te.shape

((16000, 3327),
 (4000, 3327),
 (16000, 148),
 (4000, 148),
 (16000, 4),
 (4000, 4),
 (16000, 22),
 (4000, 22))

In [ ]:
y_te.to_numpy()[0,:]

In [ ]:
Im_tr.shape

### Pre-training

In [9]:
scaler_X = MinMaxScaler().fit(X_tr) # range: 0~1
scaler_knob = MinMaxScaler().fit(knob_tr)
scaler_Im = MinMaxScaler().fit(Im_tr)
scaler_y = StandardScaler().fit(y_tr)

# X_norm_tr = torch.Tensor(scaler_X.transform(X_tr)).cuda()
# X_norm_te = torch.Tensor(scaler_X.transform(X_te)).cuda()
X_tr = torch.Tensor(X_tr).cuda()
X_te = torch.Tensor(X_te).cuda()
y_norm_tr = torch.Tensor(scaler_y.transform(y_tr)).cuda()
y_norm_te = torch.Tensor(scaler_y.transform(y_te)).cuda()

In [10]:
Im_norm_tr = torch.Tensor(scaler_Im.transform(Im_tr)).cuda()
Im_norm_te = torch.Tensor(scaler_Im.transform(Im_te)).cuda()
Dataset_tr = RocksDBDataset(X_tr, Im_norm_tr)
Dataset_te = RocksDBDataset(X_te, Im_norm_te)

loader_tr = DataLoader(dataset = Dataset_tr, batch_size = 32, shuffle=True)
loader_te = DataLoader(dataset = Dataset_te, batch_size = 32, shuffle=False)

In [11]:
k2i_model = SingleNet(input_dim=X_tr.shape[1], hidden_dim=1024, output_dim=148).cuda()

In [12]:
lr = 0.001
epochs = 30
losses_tr = []

for epoch in range(epochs):
    loss_tr = train(k2i_model, loader_tr, lr)
    losses_tr.append(loss_tr)

    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}")
        
# print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr:.4f}")

[00/30] loss_tr: 0.008892751182429492
[01/30] loss_tr: 0.0042710546874441205
[02/30] loss_tr: 0.0031172816404141488
[03/30] loss_tr: 0.002162567096296698
[04/30] loss_tr: 0.0016455949055962265
[05/30] loss_tr: 0.0013694253055145963
[06/30] loss_tr: 0.001190343408496119
[07/30] loss_tr: 0.0010547415347537026
[08/30] loss_tr: 0.0009632199752377346
[09/30] loss_tr: 0.0008759727024007589
[10/30] loss_tr: 0.0008113266212167219
[11/30] loss_tr: 0.0007535783359198831
[12/30] loss_tr: 0.0007175170193659141
[13/30] loss_tr: 0.000674728078651242
[14/30] loss_tr: 0.0006382917282753624
[15/30] loss_tr: 0.0006211926711257547
[16/30] loss_tr: 0.0005942831324646249
[17/30] loss_tr: 0.0005793707085540519
[18/30] loss_tr: 0.000553441908210516
[19/30] loss_tr: 0.0005300793341593817
[20/30] loss_tr: 0.0005186094209784642
[21/30] loss_tr: 0.0005029290979146026
[22/30] loss_tr: 0.0004919989100308158
[23/30] loss_tr: 0.0004805085865664296
[24/30] loss_tr: 0.00047077390766935424
[25/30] loss_tr: 0.0004541118

### Train with knob2vec

In [13]:
lookup_table = k2i_model.knob_fc[0].weight.T.cpu().detach().numpy()
# lookup_table = np.load('LookupTable.npy')
lookup_table.shape

(3327, 1024)

In [ ]:
# np.save('LookupTable.npy', lookup_table)

In [14]:
def get_knob2vec(data, table):
    k2vec = np.zeros((data.shape[0], 22, table.shape[1]))
    for i in range(data.shape[0]):
#         idx = (data[i]==1).nonzero().squeeze().cpu().detach().numpy()
        idx = (data[i]==1).nonzero().squeeze().cpu().detach().numpy()
        k2vec[i] = lookup_table[idx]
    return k2vec

In [15]:
K2vec_tr = torch.Tensor(get_knob2vec(X_tr, lookup_table)).cuda()
K2vec_tr = torch.reshape(K2vec_tr, (K2vec_tr.shape[0], -1))
K2vec_te = torch.Tensor(get_knob2vec(X_te, lookup_table)).cuda()
K2vec_te = torch.reshape(K2vec_te, (K2vec_te.shape[0], -1))

Dataset_K2vec_tr = RocksDBDataset(K2vec_tr, y_norm_tr)
Dataset_K2vec_te = RocksDBDataset(K2vec_te, y_norm_te)

loader_K2vec_tr = DataLoader(dataset = Dataset_K2vec_tr, batch_size = 32, shuffle=True)
loader_K2vec_te = DataLoader(dataset = Dataset_K2vec_te, batch_size = 32, shuffle=True)

/home/jieun/anaconda3/envs/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  """


In [16]:
model = SingleNet(input_dim=K2vec_tr.shape[-1], hidden_dim=64, output_dim=4).cuda()

In [17]:
lr = 0.001
epochs = 30
losses_tr = []
losses_te = []
for epoch in range(epochs):
    loss_tr = train(model, loader_K2vec_tr, lr)
    loss_te, outputs = valid(model, loader_K2vec_te)
    
    losses_tr.append(loss_tr)
    losses_te.append(loss_te)
    
    print(f'[{epoch}/{epochs}] loss_tr: {loss_tr:.8f}\tloss_te:{loss_te:.8f}')

[0/30] loss_tr: 0.17557958	loss_te:0.18121772
[1/30] loss_tr: 0.12320790	loss_te:0.18437419
[2/30] loss_tr: 0.10532429	loss_te:0.17766876
[3/30] loss_tr: 0.09626329	loss_te:0.19151275
[4/30] loss_tr: 0.08872125	loss_te:0.19363469
[5/30] loss_tr: 0.08239687	loss_te:0.19791796
[6/30] loss_tr: 0.07563802	loss_te:0.19482084
[7/30] loss_tr: 0.07166722	loss_te:0.19370911
[8/30] loss_tr: 0.06718028	loss_te:0.19548641
[9/30] loss_tr: 0.06371107	loss_te:0.19459494
[10/30] loss_tr: 0.05993534	loss_te:0.20321043
[11/30] loss_tr: 0.05624892	loss_te:0.19465362
[12/30] loss_tr: 0.05402923	loss_te:0.20398066
[13/30] loss_tr: 0.05185005	loss_te:0.20488734
[14/30] loss_tr: 0.04932535	loss_te:0.20031519
[15/30] loss_tr: 0.04780192	loss_te:0.21007408
[16/30] loss_tr: 0.04649080	loss_te:0.19928752
[17/30] loss_tr: 0.04514992	loss_te:0.20402052
[18/30] loss_tr: 0.04348185	loss_te:0.20264021
[19/30] loss_tr: 0.04165939	loss_te:0.20418413
[20/30] loss_tr: 0.04064165	loss_te:0.21427403
[21/30] loss_tr: 0.0394

In [30]:
pred = np.round(scaler_y.inverse_transform(outputs.cpu().detach().numpy()),2)
true = y_te.to_numpy()

In [29]:
from sklearn.metrics import r2_score

In [33]:
for i in range(10):
    print(f'predict rslt: {pred[i]}')
    print(f'ground truth: {true[i]}\n')

predict rslt: [13.22  6.66  5.16 98.16]
ground truth: [12.4   8.25  6.6  81.11]

predict rslt: [ 4.6  23.21  7.34 24.46]
ground truth: [ 4.4  23.04  7.7  97.67]

predict rslt: [ 6.01 17.59  5.22 61.35]
ground truth: [11.4   8.99  5.1  97.64]

predict rslt: [ 7.27 14.33  4.74 99.03]
ground truth: [ 5.6  18.24  6.   75.3 ]

predict rslt: [ 3.92 28.11  7.38 15.52]
ground truth: [17.    6.01  8.   46.08]

predict rslt: [12.97  7.58  4.51 96.72]
ground truth: [12.7   8.08  6.7  84.06]

predict rslt: [ 6.69 17.47  7.06 68.59]
ground truth: [ 4.6  22.11  7.   44.21]

predict rslt: [ 9.6  14.14  5.88 97.78]
ground truth: [ 4.   25.34  6.8  50.02]

predict rslt: [ 3.82 25.43  6.43 37.47]
ground truth: [ 8.   12.84  6.8  97.7 ]

predict rslt: [10.83  8.87  5.35 84.41]
ground truth: [ 4.7  21.56  5.1  52.98]



In [32]:
true, pred

(array([[12.4 ,  8.25,  6.6 , 81.11],
        [ 4.4 , 23.04,  7.7 , 97.67],
        [11.4 ,  8.99,  5.1 , 97.64],
        ...,
        [19.3 ,  5.31,  8.3 , 52.58],
        [22.4 ,  4.56,  6.9 , 70.29],
        [ 3.4 , 30.45,  6.2 , 15.88]]),
 array([[13.22,  6.66,  5.16, 98.16],
        [ 4.6 , 23.21,  7.34, 24.46],
        [ 6.01, 17.59,  5.22, 61.35],
        ...,
        [10.64, 11.73,  5.81, 98.46],
        [ 9.21, 12.75,  6.4 , 96.19],
        [11.48,  9.32,  5.85, 98.66]], dtype=float32))

In [31]:
r2_score(true, pred)

-0.9423747994055933

### Train with knob2vec and GRU

In [ ]:
K2vec_tr = torch.Tensor(get_knob2vec(X_tr, lookup_table)).cuda()
K2vec_te = torch.Tensor(get_knob2vec(X_te, lookup_table)).cuda()

Dataset_K2vec_tr = RocksDBDataset(K2vec_tr, y_norm_tr)
Dataset_K2vec_te = RocksDBDataset(K2vec_te, y_norm_te)

loader_K2vec_tr = DataLoader(dataset = Dataset_K2vec_tr, batch_size = 32, shuffle=True)
loader_K2vec_te = DataLoader(dataset = Dataset_K2vec_te, batch_size = 32, shuffle=True)

In [ ]:
encoder = EncoderRNN(hidden_dim=K2vec_tr.shape[-1]).cuda()
decoder = DecoderRNN(hidden_dim=K2vec_tr.shape[-1], output_dim=1).cuda()

In [ ]:
lr = 0.001
epochs = 30
losses_tr = []
losses_te = []
for epoch in range(epochs):
    loss_tr = trainRNN(encoder, decoder, loader_K2vec_tr, lr)
    loss_te = validRNN(encoder, decoder, loader_K2vec_te)
    
    losses_tr.append(loss_tr)
    losses_te.append(loss_te)
    
    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}\tloss_te:{loss_te:.4f}")

### Train with raw knob

In [ ]:
knob_norm_tr = torch.Tensor(scaler_knob.transform(knob_tr)).cuda()
knob_norm_te = torch.Tensor(scaler_knob.transform(knob_te)).cuda()

Dataset_knob_tr = RocksDBDataset(knob_norm_tr, y_norm_tr)
Dataset_knob_te = RocksDBDataset(knob_norm_te, y_norm_te)

loader_knob_tr = DataLoader(dataset = Dataset_knob_tr, batch_size = 32, shuffle=True)
loader_knob_te = DataLoader(dataset = Dataset_knob_te, batch_size = 32, shuffle=True)

In [ ]:
knob_model = SingleNet(input_dim=knob_norm_tr.shape[1], hidden_dim=16, output_dim=4).cuda()

In [ ]:
lr = 0.001
epochs = 30
losses_tr = []
losses_te = []
for epoch in range(epochs):
    loss_tr = train(knob_model, loader_knob_tr, lr)
    loss_te = valid(knob_model, loader_knob_te)
    
    losses_tr.append(loss_tr)
    losses_te.append(loss_te)
    
    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}\tloss_te:{loss_te:.4f}")

In [ ]:
pd.DataFrame(scaler_y.inverse_transform(y_norm_te.cpu().detach().numpy()))

In [ ]:
pd.DataFrame(scaler_y.inverse_transform(model(K2vec_te).cpu().detach().numpy()))

In [ ]:
_30 = scaler_y.inverse_transform(knob_model(knob_norm_te).cpu().detach().numpy()) # 30
_30

In [ ]:
pd.DataFrame(_30)

In [ ]:
pd.DataFrame(scaler_y.inverse_transform(knob_model(knob_norm_te).cpu().detach().numpy())) # 60